In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pybaseball
import seaborn as sns

/Users/planteatre/personal-projects/pitch-predictor/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# from pybaseball import statcast
# data = statcast(start_dt='2024-03-20', end_dt='2024-09-30')
# data.to_csv('savant_data_2024.csv', index=False)

In [3]:
data = pd.read_csv('savant_data_2024.csv')
data = data.iloc[::-1].copy().reset_index(drop=True)

In [4]:
data[['game_date', 'player_name', 'pitch_type', 'inning', 'balls', 'strikes', 'home_team', 'away_team']]

,game_date,player_name,pitch_type,inning,balls,strikes,home_team,away_team
0,2024-03-20,"Darvish, Yu",FF,1,0,0,SD,LAD
1,2024-03-20,"Darvish, Yu",FF,1,1,0,SD,LAD
2,2024-03-20,"Darvish, Yu",SI,1,1,1,SD,LAD
3,2024-03-20,"Darvish, Yu",SI,1,2,1,SD,LAD
4,2024-03-20,"Darvish, Yu",FF,1,0,0,SD,LAD
...,...,...,...,...,...,...,...,...
730053,2024-09-30,"Díaz, Edwin",SL,9,0,0,ATL,NYM
730054,2024-09-30,"Díaz, Edwin",SL,9,0,1,ATL,NYM
730055,2024-09-30,"Díaz, Edwin",SL,9,1,1,ATL,NYM
730056,2024-09-30,"Díaz, Edwin",SL,9,1,2,ATL,NYM


In [5]:
print(data.columns.tolist())

['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

stand - side of plate pitcher is standing (right or left) -> OHE
inning_topbot - top or bottom
p_throws - left or right handed pitcher
description - short description of the result of the pitch
des - long description of the play

In [6]:
non_numeric_cols = data.select_dtypes(exclude=['number']).columns
for col in non_numeric_cols:
    print(f"{col}: {data[col].dtype}")

pitch_type: object
game_date: object
player_name: object
events: object
description: object
des: object
game_type: object
stand: object
p_throws: object
home_team: object
away_team: object
type: object
bb_type: object
inning_topbot: object
pitch_name: object
if_fielding_alignment: object
of_fielding_alignment: object


In [7]:
filtered_data = data.drop(columns=['events', 'des', 'if_fielding_alignment', 'of_fielding_alignment', 'umpire', 'sv_id',
    'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated',
    'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
    'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'age_pit_legacy', 'age_bat_legacy',
    'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle', 'attack_angle', 'attack_direction', 
    'swing_path_tilt', 'intercept_ball_minus_batter_pos_x_inches', 'intercept_ball_minus_batter_pos_y_inches',
    'estimated_slg_using_speedangle', 'delta_pitcher_run_exp', 'hyper_speed', 'post_fld_score', 'game_type', 
    
    # following have the potential to be included to improve accuracy but requires further feature engineering
    
    'release_pos_x', 'release_pos_z', 'spin_dir', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y', 
    'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'effective_speed', 'release_spin_rate', 'release_extension',
    'babip_value', 'iso_value', 'launch_speed_angle', 'post_away_score', 'post_home_score', 'post_bat_score',
    'delta_home_win_exp', 'delta_run_exp', 'bat_speed', 'swing_length', 'home_score_diff', 'bat_score_diff', 'home_win_exp', 
    'bat_win_exp', 'sz_top', 'sz_bot', 'woba_value', 'woba_denom', 'age_pit', 'age_bat', 'hit_distance_sc', 'spin_axis',

    'game_year'
    
])

In [8]:
non_numeric_cols = filtered_data.select_dtypes(exclude=['number']).columns
for col in non_numeric_cols:
    print(f"{col}: {filtered_data[col].dtype}")

pitch_type: object
game_date: object
player_name: object
description: object
stand: object
p_throws: object
home_team: object
away_team: object
type: object
bb_type: object
inning_topbot: object
pitch_name: object


In [9]:
data_more_filtered = filtered_data.copy()#.drop(columns=['pitch_name', 'home_team', 'away_team', 'player_name', 'game_date'])

In [10]:
print(data_more_filtered.select_dtypes(exclude=['number']).columns)

Index(['pitch_type', 'game_date', 'player_name', 'description', 'stand',
       'p_throws', 'home_team', 'away_team', 'type', 'bb_type',
       'inning_topbot', 'pitch_name'],
      dtype='object')


### Feature Engineering -- Previous Pitch Data

treat zone, hit_location, on_Xb as categorical data

In [11]:
# idea from https://seanjhannon.medium.com/baseball-pitch-prediction-with-deep-learning-df68094fcc65
data_more_filtered.loc[:,'plate_app_id'] =  data_more_filtered['game_pk'].astype(str) + data_more_filtered['batter'].astype(str) + data_more_filtered['at_bat_number'].astype(str)


In [12]:
# modifies data to boolean values for on_3b, on_2b, on_1b
for base_col in ['on_3b', 'on_2b', 'on_1b']:
    data_more_filtered[base_col] = data_more_filtered[base_col].apply(
        lambda x: 0 if (isinstance(x, (int, float)) and not pd.isnull(x) and x == 0)
        else (1 if isinstance(x, (int, float)) and not pd.isnull(x) else 0)
    )

data_more_filtered[['plate_app_id', 'on_3b', 'on_2b', 'on_1b']]

,plate_app_id,on_3b,on_2b,on_1b
0,7454446051411,0,0,0
1,7454446051411,0,0,0
2,7454446051411,0,0,0
3,7454446051411,0,0,0
4,7454446602712,0,0,1
...,...,...,...,...
730053,74713951859582,0,1,0
730054,74713951859582,0,1,0
730055,74713951859582,0,1,0
730056,74713951859582,0,1,0


In [13]:
# Check if 'stand' or 'p_throws' can change within the same plate_app_id
stand_varies_mask = data_more_filtered.groupby('plate_app_id')['stand'].nunique() > 1
p_throws_varies_mask = data_more_filtered.groupby('plate_app_id')['p_throws'].nunique() > 1

stand_varies_count = stand_varies_mask.sum()
p_throws_varies_count = p_throws_varies_mask.sum()

print(f"Number of plate_app_id where 'stand' varies: {stand_varies_count}")
print(f"Number of plate_app_id where 'p_throws' varies: {p_throws_varies_count}")

# Optionally, show some example plate_app_id values if they do vary
if stand_varies_count > 0:
    print("Example plate_app_id(s) where 'stand' varies:")
    print(stand_varies_mask[stand_varies_mask].index.tolist()[:5])
if p_throws_varies_count > 0:
    print("Example plate_app_id(s) where 'p_throws' varies:")
    print(p_throws_varies_mask[p_throws_varies_mask].index.tolist()[:5])

# varies too little to be useful as pre-pitch data for the following pitch


Number of plate_app_id where 'stand' varies: 6
Number of plate_app_id where 'p_throws' varies: 31
Example plate_app_id(s) where 'stand' varies:
['74479568308323', '74507867076430', '74570760807034', '74657259387156', '74783466613425']
Example plate_app_id(s) where 'p_throws' varies:
['74479568308323', '74489267639155', '7450075437604', '74502167127742', '74505168299875']


In [14]:
# Calculate the difference in away_score and home_score within each plate_app_id
away_score_diff = data_more_filtered.groupby('plate_app_id')['away_score'].diff().fillna(0)
home_score_diff = data_more_filtered.groupby('plate_app_id')['home_score'].diff().fillna(0)

# Count the number of times both away_score_diff and home_score_diff are greater than 0 at the same time
both_scores_increased = ((away_score_diff > 0) & (home_score_diff > 0)).sum()

print(f"Number of times both away_score and home_score increased: {both_scores_increased}")

# Create a new column 'prev_runs_scored' that is simply the sum of away_score_diff and home_score_diff
# since there is no data weirdness where both away and home score increase, we can add the two series
data_more_filtered['prev_runs_scored'] = away_score_diff + home_score_diff




Number of times both away_score and home_score increased: 0


In [15]:
data_more_filtered.columns

Index(['pitch_type', 'game_date', 'release_speed', 'player_name', 'batter',
       'pitcher', 'description', 'zone', 'stand', 'p_throws', 'home_team',
       'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes',
       'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot',
       'launch_speed', 'launch_angle', 'game_pk', 'release_pos_y',
       'at_bat_number', 'pitch_number', 'pitch_name', 'home_score',
       'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher',
       'n_priorpa_thisgame_player_at_bat', 'pitcher_days_since_prev_game',
       'batter_days_since_prev_game', 'pitcher_days_until_next_game',
       'batter_days_until_next_game', 'plate_app_id', 'prev_runs_scored'],
      dtype='object')

In [16]:
# data to shift from previous pitch(es) in the same plate appearance
# i.e. not known pre-pitch
memory_features = ['pitch_type', 'description', 'zone', 'release_speed', 'type', 'hit_location', 'bb_type',
        'launch_speed', 'balls', 'strikes', 'launch_angle', 'release_pos_y', ]

# known pre-pitch
context_features = ['on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'at_bat_number', 'pitch_number',
                    'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat', 
                    'pitcher_days_since_prev_game', 'batter_days_since_prev_game', 'pitcher_days_until_next_game', 
                    'batter_days_until_next_game', 'prev_runs_scored']

In [17]:
data_more_filtered['n_priorpa_thisgame_player_at_bat'].value_counts()

n_priorpa_thisgame_player_at_bat
0    202067
1    179495
2    166912
3    139261
4     39596
5      2653
6        74
Name: count, dtype: int64

### Encoding Categorical Variables

In [18]:
for col in data_more_filtered.select_dtypes(exclude=['number']).columns:
    print(f"{col}: {len(data_more_filtered[col].unique())} unique values")

pitch_type: 18 unique values
game_date: 190 unique values
player_name: 1082 unique values
description: 13 unique values
stand: 2 unique values
p_throws: 2 unique values
home_team: 30 unique values
away_team: 30 unique values
type: 3 unique values
bb_type: 5 unique values
inning_topbot: 2 unique values
pitch_name: 18 unique values
plate_app_id: 189151 unique values


In [19]:
data_more_filtered['description'].value_counts()

description
ball                       242247
foul                       131539
hit_into_play              128483
called_strike              119469
swinging_strike             77876
blocked_ball                15265
foul_tip                     7508
swinging_strike_blocked      4036
hit_by_pitch                 2103
foul_bunt                    1255
missed_bunt                   206
pitchout                       55
bunt_foul_tip                  16
Name: count, dtype: int64

In [20]:
data_more_filtered.loc[
    data_more_filtered['description'].isin(['missed_bunt', 'bunt_foul_tip']),
    'description'
] = 'foul_bunt'

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Only encode the specified categorical columns, zone and hit_location are numbers but categorical
cols_to_encode = ['description', 'stand', 'p_throws', 'type', 'bb_type', 'inning_topbot', 'zone', 'hit_location']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='if_binary')
encoded = encoder.fit_transform(data_more_filtered[cols_to_encode])

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cols_to_encode), index=data_more_filtered.index)
filtered_data_encoded = pd.concat([data_more_filtered.drop(columns=cols_to_encode), encoded_df], axis=1)



In [38]:
data_more_filtered['hit_location'].value_counts()

hit_location
2.0    44423
8.0    22508
9.0    20545
7.0    19811
6.0    15861
4.0    14666
5.0    13750
3.0     9497
1.0     4558
Name: count, dtype: int64

In [32]:
filtered_data_encoded

,pitch_type,game_date,release_speed,player_name,batter,pitcher,home_team,away_team,balls,strikes,...,hit_location_1.0,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,hit_location_nan
0,FF,2024-03-20,94.5,"Darvish, Yu",605141,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,FF,2024-03-20,92.6,"Darvish, Yu",605141,506433,SD,LAD,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,SI,2024-03-20,93.4,"Darvish, Yu",605141,506433,SD,LAD,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,SI,2024-03-20,93.9,"Darvish, Yu",605141,506433,SD,LAD,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,FF,2024-03-20,95.7,"Darvish, Yu",660271,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730053,SL,2024-09-30,89.1,"Díaz, Edwin",518595,621242,ATL,NYM,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730054,SL,2024-09-30,91.3,"Díaz, Edwin",518595,621242,ATL,NYM,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730055,SL,2024-09-30,91.1,"Díaz, Edwin",518595,621242,ATL,NYM,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730056,SL,2024-09-30,90.7,"Díaz, Edwin",518595,621242,ATL,NYM,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Compute the correlation matrix
corr = filtered_data_encoded.corr()

# Set up the matplotlib figure
plt.figure(figsize=(16, 12))

# Draw the heatmap
sns.heatmap(corr, cmap='coolwarm', center=0, linewidths=0.5)

plt.title('Correlation Heatmap of Encoded Filtered Data')
plt.tight_layout()
plt.show()


In [34]:
# Find pairs of features with perfect or near-perfect correlation (>|0.98|, excluding self-pairs)
threshold = 0.98

# corr = filtered_data_encoded.corr()
corr = filtered_data_encoded.select_dtypes(exclude=['object']).corr()

# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# Find pairs with correlation above threshold or below -threshold
perfect_corr_pairs = [
    (col, row, corr_value)
    for col in upper.columns
    for row, corr_value in upper[col].items()
    if abs(corr_value) >= threshold
]

# Display the pairs
for col1, col2, corr_val in perfect_corr_pairs:
    print(f"{col1} <--> {col2}: correlation = {corr_val:.3f}")


type_X <--> description_hit_into_play: correlation = 1.000
bb_type_nan <--> description_hit_into_play: correlation = -1.000
bb_type_nan <--> type_X: correlation = -1.000


In [35]:
filtered_data_encoded = filtered_data_encoded.drop(columns=['type_X', 'bb_type_nan'], errors='ignore')


In [39]:
filtered_data_encoded

,pitch_type,game_date,release_speed,player_name,batter,pitcher,home_team,away_team,balls,strikes,...,hit_location_1.0,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,hit_location_nan
0,FF,2024-03-20,94.5,"Darvish, Yu",605141,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,FF,2024-03-20,92.6,"Darvish, Yu",605141,506433,SD,LAD,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,SI,2024-03-20,93.4,"Darvish, Yu",605141,506433,SD,LAD,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,SI,2024-03-20,93.9,"Darvish, Yu",605141,506433,SD,LAD,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,FF,2024-03-20,95.7,"Darvish, Yu",660271,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730053,SL,2024-09-30,89.1,"Díaz, Edwin",518595,621242,ATL,NYM,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730054,SL,2024-09-30,91.3,"Díaz, Edwin",518595,621242,ATL,NYM,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730055,SL,2024-09-30,91.1,"Díaz, Edwin",518595,621242,ATL,NYM,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730056,SL,2024-09-30,90.7,"Díaz, Edwin",518595,621242,ATL,NYM,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [40]:
cols_to_list = ['description', 'stand', 'p_throws', 'type', 'bb_type', 'inning_topbot', 'zone', 'hit_location']
matching_columns = [col for col in filtered_data_encoded.columns if any(key in col for key in cols_to_list)]
print(matching_columns)


['pitch_type', 'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 'description_foul_bunt', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 'description_pitchout', 'description_swinging_strike', 'description_swinging_strike_blocked', 'stand_R', 'p_throws_R', 'type_B', 'type_S', 'bb_type_fly_ball', 'bb_type_ground_ball', 'bb_type_line_drive', 'bb_type_popup', 'inning_topbot_Top', 'zone_1.0', 'zone_2.0', 'zone_3.0', 'zone_4.0', 'zone_5.0', 'zone_6.0', 'zone_7.0', 'zone_8.0', 'zone_9.0', 'zone_11.0', 'zone_12.0', 'zone_13.0', 'zone_14.0', 'zone_nan', 'hit_location_1.0', 'hit_location_2.0', 'hit_location_3.0', 'hit_location_4.0', 'hit_location_5.0', 'hit_location_6.0', 'hit_location_7.0', 'hit_location_8.0', 'hit_location_9.0', 'hit_location_nan']


In [45]:
# Features known BEFORE pitch is thrown
context_features = [
    'batter', 'pitcher', 'stand_R', 'p_throws_R', 'balls', 'strikes',

    'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 'at_bat_number', 'pitch_number',
            'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat', 
            'pitcher_days_since_prev_game', 'batter_days_since_prev_game', 'pitcher_days_until_next_game', 
            'batter_days_until_next_game', 'prev_runs_scored'
]
        
# Features from PREVIOUS pitches (memory)
memory_features = [
    'pitch_type', 'release_speed', 'launch_speed', 'launch_angle', 'release_pos_y',

    'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 
    'description_foul_bunt', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 
    'description_pitchout', 'description_swinging_strike', 'description_swinging_strike_blocked', 
    'type_B', 'type_S', 'bb_type_fly_ball', 'bb_type_ground_ball', 'bb_type_line_drive', 'bb_type_popup', 
    'zone_1.0', 'zone_2.0', 'zone_3.0', 'zone_4.0', 'zone_5.0', 'zone_6.0', 'zone_7.0', 'zone_8.0', 
    'zone_9.0', 'zone_11.0', 'zone_12.0', 'zone_13.0', 'zone_14.0', 'zone_nan', 'hit_location_1.0', 'hit_location_2.0', 
    'hit_location_3.0', 'hit_location_4.0', 'hit_location_5.0', 'hit_location_6.0', 'hit_location_7.0', 'hit_location_8.0', 
    'hit_location_9.0', 'hit_location_nan'
]
len(memory_features) + len(context_features)

71

In [47]:
# List columns that aren't in memory_features or context_features
all_features = set(filtered_data_encoded.columns)
used_features = set(context_features) | set(memory_features)
unused_features = [col for col in filtered_data_encoded.columns if col not in used_features]
print("Columns not in memory_features or context_features:", unused_features)


Columns not in memory_features or context_features: ['game_date', 'player_name', 'home_team', 'away_team', 'game_pk', 'pitch_name', 'plate_app_id']


In [43]:
object_columns = filtered_data_encoded.select_dtypes(include='object').columns.tolist()
print("Object columns:", object_columns)


Object columns: ['pitch_type', 'game_date', 'player_name', 'home_team', 'away_team', 'pitch_name', 'plate_app_id']


In [44]:
len(filtered_data_encoded.columns)

78

In [49]:
# Check for any null or NaN values in the dataset
null_counts = filtered_data_encoded.isnull().sum()
print("Null values per column:")
print(null_counts[null_counts > 0])

# Also check if there are any NaNs at all
total_nulls = filtered_data_encoded.isnull().sum().sum()
print(f"Total null/NaN values in DataFrame: {total_nulls}")


Null values per column:
pitch_type                        6867
release_speed                     8070
launch_speed                    488495
launch_angle                    488178
release_pos_y                     8069
pitch_name                        6867
pitcher_days_since_prev_game     55457
batter_days_since_prev_game      28899
pitcher_days_until_next_game     48365
batter_days_until_next_game      26741
dtype: int64
Total null/NaN values in DataFrame: 1166008


In [50]:
# Drop columns with too many missing values
cols_to_drop = [
    'pitcher_days_since_prev_game',
    'batter_days_since_prev_game',
    'pitcher_days_until_next_game',
    'batter_days_until_next_game',
    'launch_speed', 'launch_angle'
]
filtered_data_encoded = filtered_data_encoded.drop(columns=cols_to_drop)

# Drop rows where pitch_type, release_speed, or release_pos_y are null
filtered_data_encoded = filtered_data_encoded.dropna(subset=['pitch_type', 'release_speed', 'release_pos_y'])


In [51]:
filtered_data_encoded

,pitch_type,game_date,release_speed,player_name,batter,pitcher,home_team,away_team,balls,strikes,...,hit_location_1.0,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,hit_location_nan
0,FF,2024-03-20,94.5,"Darvish, Yu",605141,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,FF,2024-03-20,92.6,"Darvish, Yu",605141,506433,SD,LAD,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,SI,2024-03-20,93.4,"Darvish, Yu",605141,506433,SD,LAD,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,SI,2024-03-20,93.9,"Darvish, Yu",605141,506433,SD,LAD,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,FF,2024-03-20,95.7,"Darvish, Yu",660271,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730053,SL,2024-09-30,89.1,"Díaz, Edwin",518595,621242,ATL,NYM,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730054,SL,2024-09-30,91.3,"Díaz, Edwin",518595,621242,ATL,NYM,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730055,SL,2024-09-30,91.1,"Díaz, Edwin",518595,621242,ATL,NYM,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
730056,SL,2024-09-30,90.7,"Díaz, Edwin",518595,621242,ATL,NYM,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
filtered_data_encoded.to_csv('encoded_savant_2024.csv')

In [ ]:
 # Features known BEFORE pitch is thrown
context_features = [
    'batter', 'pitcher', 'stand_R', 'p_throws_R', 'balls', 'strikes',

    'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 'at_bat_number', 'pitch_number',
    'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
    'prev_runs_scored'
]
        
# Features from PREVIOUS pitches (memory)
memory_features = [
    'pitch_type', 'release_speed', 'release_pos_y',

    'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 
    'description_foul_bunt', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 
    'description_pitchout', 'description_swinging_strike', 'description_swinging_strike_blocked', 
    'type_B', 'type_S', 'bb_type_fly_ball', 'bb_type_ground_ball', 'bb_type_line_drive', 'bb_type_popup', 
    'zone_1.0', 'zone_2.0', 'zone_3.0', 'zone_4.0', 'zone_5.0', 'zone_6.0', 'zone_7.0', 'zone_8.0', 
    'zone_9.0', 'zone_11.0', 'zone_12.0', 'zone_13.0', 'zone_14.0', 'zone_nan', 'hit_location_1.0', 'hit_location_2.0', 
    'hit_location_3.0', 'hit_location_4.0', 'hit_location_5.0', 'hit_location_6.0', 'hit_location_7.0', 'hit_location_8.0', 
    'hit_location_9.0', 'hit_location_nan'
]

In [53]:
# Check for any null or NaN values in the dataset
null_counts = filtered_data_encoded.isnull().sum()
print("Null values per column:")
print(null_counts[null_counts > 0])

# Also check if there are any NaNs at all
total_nulls = filtered_data_encoded.isnull().sum().sum()
print(f"Total null/NaN values in DataFrame: {total_nulls}")

Null values per column:
Series([], dtype: int64)
Total null/NaN values in DataFrame: 0


In [55]:
num_unique_batters = filtered_data_encoded['batter'].nunique()
num_unique_pitchers = filtered_data_encoded['pitcher'].nunique()
print(f"Number of unique batters: {num_unique_batters}")
print(f"Number of unique pitchers: {num_unique_pitchers}")
num_unique_pitch_types = filtered_data_encoded['pitch_type'].nunique()
print(f"Number of unique pitch types: {num_unique_pitch_types}")



Number of unique batters: 971
Number of unique pitchers: 974
Number of unique pitch types: 17


In [58]:
# Map pitch_type to pitch_type_group
pitch_type_map = {
    'FF': 'FAST', 'SI': 'FAST', 'FC': 'FAST',
    'CH': 'OFF', 'FS': 'OFF', 'FO': 'OFF',
    'CU': 'BREAK', 'KC': 'BREAK', 'CS': 'BREAK', 'KN': 'BREAK', 'SL': 'BREAK', 'ST': 'BREAK', 'SV': 'BREAK',
    'EP': 'OTH', 'FA': 'OTH', 'IN': 'OTH', 'PO': 'OTH'
}

filtered_data_encoded['pitch_type_group'] = filtered_data_encoded['pitch_type'].map(pitch_type_map)
print(filtered_data_encoded['pitch_type_group'].value_counts(dropna=False))


pitch_type_group
FAST     403091
BREAK    221339
OFF       96141
OTH        1262
NaN         155
Name: count, dtype: int64


In [64]:
filtered_data_encoded = filtered_data_encoded.dropna(subset=['pitch_type_group'])


In [65]:
from sklearn.preprocessing import StandardScaler

# Select columns to normalize
cols_to_normalize = ['release_speed', 'release_pos_y']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the selected columns
filtered_data_encoded[cols_to_normalize] = scaler.fit_transform(filtered_data_encoded[cols_to_normalize])

# Check the result
print(filtered_data_encoded[cols_to_normalize].describe())

       release_speed  release_pos_y
count   7.218330e+05   7.218330e+05
mean   -1.614349e-15  -1.831278e-14
std     1.000001e+00   1.000001e+00
min    -9.580467e+00  -5.000071e+00
25%    -7.111089e-01  -6.420656e-01
50%     1.423577e-01  -2.887896e-02
75%     8.117433e-01   6.281067e-01
max     2.736227e+00   7.154164e+00


/var/folders/rb/k_y0ysf94m7_k0f10k0f4h5r0000gn/T/ipykernel_90252/2258328032.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_encoded[cols_to_normalize] = scaler.fit_transform(filtered_data_encoded[cols_to_normalize])


In [66]:
filtered_data_encoded.to_csv('final_encoded_savant_2024.csv')

In [69]:
data[['player_name', 'pitcher', 'batter']]

,player_name,pitcher,batter
0,"Darvish, Yu",506433,605141
1,"Darvish, Yu",506433,605141
2,"Darvish, Yu",506433,605141
3,"Darvish, Yu",506433,605141
4,"Darvish, Yu",506433,660271
...,...,...,...
730053,"Díaz, Edwin",621242,518595
730054,"Díaz, Edwin",621242,518595
730055,"Díaz, Edwin",621242,518595
730056,"Díaz, Edwin",621242,518595


In [78]:
import numpy as np

# List of context features from model.py
context_features = [
    'stand_R', 'p_throws_R', 'balls', 'strikes',
    'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 'at_bat_number', 'pitch_number',
    'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
    'prev_runs_scored'
]

# Sample a random row with these features from filtered_data_encoded
list(filtered_data_encoded.iloc[100][context_features].values)


[np.float64(0.0),
 np.float64(1.0),
 np.int64(0),
 np.int64(2),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(3),
 np.float64(0.0),
 np.int64(23),
 np.int64(3),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.float64(0.0)]

In [79]:
filtered_data_encoded

,pitch_type,game_date,release_speed,player_name,batter,pitcher,home_team,away_team,balls,strikes,...,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,hit_location_nan,pitch_type_group
0,FF,2024-03-20,0.895416,"Darvish, Yu",605141,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FAST
1,FF,2024-03-20,0.577458,"Darvish, Yu",605141,506433,SD,LAD,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FAST
2,SI,2024-03-20,0.711335,"Darvish, Yu",605141,506433,SD,LAD,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FAST
3,SI,2024-03-20,0.795009,"Darvish, Yu",605141,506433,SD,LAD,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FAST
4,FF,2024-03-20,1.096232,"Darvish, Yu",660271,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,FAST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730053,SL,2024-09-30,-0.008254,"Díaz, Edwin",518595,621242,ATL,NYM,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,BREAK
730054,SL,2024-09-30,0.359908,"Díaz, Edwin",518595,621242,ATL,NYM,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,BREAK
730055,SL,2024-09-30,0.326439,"Díaz, Edwin",518595,621242,ATL,NYM,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,BREAK
730056,SL,2024-09-30,0.259500,"Díaz, Edwin",518595,621242,ATL,NYM,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,BREAK


In [100]:
import pandas as pd
df = pd.read_csv('final_encoded_savant_2024.csv')




In [101]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode 'pitch_type_group' column
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='if_binary')
pitch_type_group_encoded = encoder.fit_transform(df[['pitch_type_group']])

# Get the new column names
encoded_col_names = [f'pitch_type_group_{cat}' for cat in encoder.categories_[0]]

# Add the new columns to df
for i, col in enumerate(encoded_col_names):
    df[col] = pitch_type_group_encoded[:, i]

# Optionally, drop the original column if you don't need it
df = df.drop(columns=['pitch_type_group_OTH', 'Unnamed: 0'])


In [102]:
df

,pitch_type,game_date,release_speed,player_name,batter,pitcher,home_team,away_team,balls,strikes,...,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,hit_location_nan,pitch_type_group,pitch_type_group_BREAK,pitch_type_group_FAST,pitch_type_group_OFF
0,FF,2024-03-20,0.895416,"Darvish, Yu",605141,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,FAST,0.0,1.0,0.0
1,FF,2024-03-20,0.577458,"Darvish, Yu",605141,506433,SD,LAD,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,FAST,0.0,1.0,0.0
2,SI,2024-03-20,0.711335,"Darvish, Yu",605141,506433,SD,LAD,1,1,...,0.0,0.0,0.0,0.0,0.0,1.0,FAST,0.0,1.0,0.0
3,SI,2024-03-20,0.795009,"Darvish, Yu",605141,506433,SD,LAD,2,1,...,0.0,0.0,0.0,0.0,0.0,1.0,FAST,0.0,1.0,0.0
4,FF,2024-03-20,1.096232,"Darvish, Yu",660271,506433,SD,LAD,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,FAST,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721828,SL,2024-09-30,-0.008254,"Díaz, Edwin",518595,621242,ATL,NYM,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,BREAK,1.0,0.0,0.0
721829,SL,2024-09-30,0.359908,"Díaz, Edwin",518595,621242,ATL,NYM,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,BREAK,1.0,0.0,0.0
721830,SL,2024-09-30,0.326439,"Díaz, Edwin",518595,621242,ATL,NYM,1,1,...,0.0,0.0,0.0,0.0,0.0,1.0,BREAK,1.0,0.0,0.0
721831,SL,2024-09-30,0.259500,"Díaz, Edwin",518595,621242,ATL,NYM,1,2,...,0.0,0.0,0.0,0.0,0.0,1.0,BREAK,1.0,0.0,0.0


In [103]:
df.to_csv('final_final_encoded_savant_2024.csv')

In [104]:
print("Data types of context_features elements:")
for i, val in enumerate(filtered_data_encoded.iloc[100][context_features].values):
    print(f"Index {i}: {type(val)}")


Data types of context_features elements:
Index 0: <class 'numpy.float64'>
Index 1: <class 'numpy.float64'>
Index 2: <class 'numpy.int64'>
Index 3: <class 'numpy.int64'>
Index 4: <class 'numpy.int64'>
Index 5: <class 'numpy.int64'>
Index 6: <class 'numpy.int64'>
Index 7: <class 'numpy.int64'>
Index 8: <class 'numpy.int64'>
Index 9: <class 'numpy.float64'>
Index 10: <class 'numpy.int64'>
Index 11: <class 'numpy.int64'>
Index 12: <class 'numpy.int64'>
Index 13: <class 'numpy.int64'>
Index 14: <class 'numpy.int64'>
Index 15: <class 'numpy.int64'>
Index 16: <class 'numpy.int64'>
Index 17: <class 'numpy.int64'>
Index 18: <class 'numpy.float64'>
